In [ ]:
%pip install transformers sentencepiece datasets

In [11]:
from datasets import load_dataset
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from tqdm import tqdm_notebook
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AdamW, get_linear_schedule_with_warmup, get_linear_schedule_with_warmup

sns.set()

In [17]:
# Load Model Checkpoint
modelCheckpoint = 'google/mt5-base'

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(modelCheckpoint)

# Load Model
model = AutoModelForSeq2SeqLM.from_pretrained(modelCheckpoint)
model = model.cuda()

C:\Users\mekae\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [18]:
# Make sure model inference is working

input = "I been DOING this for most my LIFE with no ADVICE"

# Tokenize
tokenIDs = tokenizer.encode(input)

tokenIDs

[336,
 2101,
 9498,
 9491,
 714,
 332,
 2250,
 1037,
 84145,
 514,
 375,
 104429,
 45533,
 1]